In [ ]:
#pip install stop-words faiss-cpu

# Set-up

In [ ]:
#conda install -n .conda ipykernel --update-deps --force-reinstall

In [ ]:
#conda env create -f environment.yml

Imports packages

In [ ]:
### DATABASES
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

### SCRAPING
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

### VISUALISATION
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

### DIVERS
from tqdm.notebook import tqdm

### DATACLEANING
from collections import Counter

### TEXT CLEANING
from nltk.corpus import stopwords
from stop_words import get_stop_words
import nltk
nltk.download('stopwords')
import re

### NLP
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss

In [ ]:
selected_websites = [
    "https://www.droit-compta-gestion.fr",
    "https://www.entreprendre-maintenant.fr"
]
output_dir = 'output'

In [ ]:
#pip install jupyter ipywidgets

Téléchargement du modèle pour Spacy (NLP)

In [ ]:
# Sélection du modèle (modèle moyen de SpaCy) # ou md 
!spacy download fr_core_news_sm 
nlp = spacy.load("fr_core_news_sm") #ou md

Imports des Dataframes

In [ ]:
# Importation des données
df = pd.read_csv('input/df_final.csv', sep='|')
# Pour chaque site, on regroupe les contenus
grouped = df[['website','article_canonical_url', 'article_content']]

Définition des stop-words

In [ ]:
# Liste des prépositions
prepositions_fr = [
    "à", "après", "avant", "avec", "chez", "contre", "dans", "de", "depuis",
    "derrière", "devant", "durant", "en", "entre", "envers", "excepté", "hors",
    "jusque", "malgré", "par", "parmi", "pendant", "pour", "près", "sans",
    "sauf", "selon", "sous", "suivant", "sur", "vers", "via", "concernant",
    "outre", "quant à", "auprès de", "autour de", "à travers", "au-dessus de",
    "au-dessous de", "au-delà de", "en dehors de", "en face de", "envers",
    "grâce à", "loin de", "près de", "quant à"
]

# Autres mots à éliminer
stop_words_others = [
    "nan", "BIEN", "Cas", "faut", "Non", "Lire", "Plan", "ligne", "prendre", 
    "Choisir", "devez", "choix", "Place", "grâce", 'url="http://www.comptazine.fr', 
    "ça", "de france", "considérez", "post views", "ce", "contents1", 
    "quelles", "ii", "toogle", "a", "b", "c", "d", "e", "f","g", "h", 
    "i", "k", "j", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", 
    "v", "w", "x", "y", "z", "plan de l'articleles", "choisissez", 
    "plan de l'articlequ'", 'icon="icon', "plan de l'", "navigation –",
    "bts cg – cours", "fr", "px1", "processus 2", "processus-2-cours-bts", 
    "jusqu'", "n'hésitez", "utilisez", "pensez", "prenez", "puisqu'"
]

# Liste des stopwords français
stop_words_fr = set(stopwords.words('french')) | set(get_stop_words('french')) | set(prepositions_fr) | set(nlp.Defaults.stop_words) | set(stop_words_others)
stop_words_fr = set(w.lower() for w in stop_words_fr)



Comptage des mots-clés

In [ ]:
# Comptage des mots-clés
def compter_mots_cles(text):
    words = text.lower().split()
    words_filtrés = [word for word in words if word.isalpha() and word not in stop_words_fr]
    compteur = Counter(words_filtrés)
    return compteur
df['keyword_count'] = df['article_content'].astype(str).apply(compter_mots_cles)

In [ ]:
df

In [ ]:
# DataFrame long
rows = [
    {'article_url': row.article_url, 'keyword': k, 'frequence': v}
    for row in df.itertuples()
    for k, v in row.keyword_count.items()
]
df_keywords = pd.DataFrame(rows)

In [ ]:
df_keywords

In [ ]:
texts = df['article_content'].astype(str).tolist()
entities_list = []
entities_freq_list = []

In [ ]:
print("Traitement en cours, la barre de progression va s'afficher après le premier batch…")

for doc in tqdm(nlp.pipe(texts, batch_size=125, n_process=18), total=len(texts), desc="Extraction des entités avec SpaCy"):
    ents = []
    for ent in doc.ents:
        text_lower = ent.text.lower()
        if text_lower not in stop_words_fr:
            ents.append(text_lower)
    entities_list.append(ents)
    entities_freq_list.append(dict(Counter(ents)))
    
df['entities'] = entities_list
df['entities_freq'] = entities_freq_list

In [ ]:
# DataFrame des entités nommées
df_entities = pd.DataFrame([
    {'website': row.website, 'article_url': row.article_url, 'terme': k, 'frequence': v}
    for row in df.itertuples()
    for k, v in row.entities_freq.items()
])
df_entities['type'] = 'entité'

In [ ]:
# DataFrame des mots-clés
df_keywords = pd.DataFrame([
    {'website': row.website, 'article_url': row.article_url, 'terme': k, 'frequence': v}
    for row in df.itertuples()
    for k, v in row.keyword_count.items()
])
df_keywords['type'] = 'mot-clé'

In [ ]:
df_termes = pd.concat([df_entities, df_keywords], ignore_index=True)
df_termes["terme"] = df_termes["terme"].str.lower()

In [ ]:
df_termes.to_csv(f'{output_dir}/df_termes.csv', sep='|')

In [ ]:
# Fonction de préprocessing équivalente à celle utilisée par TfidfVectorizer
def sklearn_tokenizer(text):
    text = text.lower()
    tokens = re.findall(r"\b\w\w+\b", text)  # mots de 2+ lettres uniquement
    return tokens

# Nettoyage de tes stop_words selon le tokenizer de sklearn
stop_words_clean = set()
for word in stop_words_fr:
    tokens = sklearn_tokenizer(str(word))
    stop_words_clean.update(tokens)

# Appliquer au vectorizer
vectorizer = TfidfVectorizer(stop_words=stop_words_clean, max_features=1000)

# 1. TF-IDF
vectorizer = TfidfVectorizer(stop_words=list(stop_words_fr), max_features=1000)
tfidf_sparse = vectorizer.fit_transform(df['article_content'].astype(str))

# 2. Conversion en dense et float32
tfidf_matrix = tfidf_sparse.toarray().astype(np.float32)

# 3. Normalisation L2
faiss.normalize_L2(tfidf_matrix)

# 4. Index FAISS
index = faiss.IndexFlatIP(tfidf_matrix.shape[1])
index.add(tfidf_matrix)

# 5. Recherche des voisins
batch_size = 1000
D_list, I_list = [], []
for i in tqdm(range(0, tfidf_matrix.shape[0], batch_size), desc="Recherche FAISS"):
    D_batch, I_batch = index.search(tfidf_matrix[i:i+batch_size], k=6)
    D_list.append(D_batch)
    I_list.append(I_batch)
D = np.vstack(D_list)
I = np.vstack(I_list)

# 6. Création des liens
links = []
for idx, (neighbors, sims) in tqdm(enumerate(zip(I, D)), total=len(df), desc="Détection des liens internes"):
    for neighbor_idx, sim in zip(neighbors[1:], sims[1:]):  # [1:] pour ignorer le self-match
        links.append({
            'article_website': df.iloc[idx]['website'],
            'source_url': df.iloc[idx]['article_url'],
            'target_url': df.iloc[neighbor_idx]['article_url'],
            'similarity_score': sim * 100  # déjà normalisé
        })

df_links = pd.DataFrame(links)
df_links = df_links[df_links['similarity_score'] != 0]

In [ ]:
def get_liens_internes(html, site):
    soup = BeautifulSoup(html, "html.parser")
    liens = {}
    for a in soup.find_all('a', href=True):
        href = a.get('href', '')
        if href.startswith(site):
            liens[href.rstrip('/')] = a.get_text(strip=True)
    return liens


url_to_html = dict(zip(df['article_url'], df['article_raw_content']))
url_to_site = dict(zip(df['article_url'], df['website']))


liens_internes_dict = {
    url: get_liens_internes(str(html), url_to_site.get(url, ""))
    for url, html in tqdm(url_to_html.items(), desc="Extraction des liens internes")
}

# 2. Vérification rapide pour chaque suggestion de lien
df_links['lien_existant'] = [
    row['target_url'] in liens_internes_dict.get(row['source_url'], set())
    for _, row in tqdm(df_links.iterrows(), total=len(df_links), desc="Vérification des liens existants")
]

# récupération des liens existants
anchor_texts = []
lien_existants = []

for _, row in tqdm(df_links.iterrows(), total=len(df_links), desc="Mapping ancres / liens"):
    liens = liens_internes_dict.get(row['source_url'], {})
    anchor = liens.get(row['target_url'].rstrip('/'))
    anchor_texts.append(anchor)
    lien_existants.append(anchor is not None)

df_links['lien_existant'] = lien_existants
df_links['anchor_text'] = anchor_texts


In [ ]:
df_internal_links = df_links[
    df_links.apply(lambda row: str(row['target_url']).startswith(str(row['article_website'])), axis=1)
]
df_internal_links = df_internal_links[df_internal_links['target_url'] != df_internal_links['source_url']]

In [ ]:
df_internal_links['lien_existant'].sum()

In [ ]:
df_internal_links.reset_index(inplace=True, drop=True)

In [ ]:
df_internal_links.to_csv(f'{output_dir}/df_internal_links.csv', sep='|')

In [ ]:
df_valid = df_internal_links[df_internal_links['lien_existant'] == True]

df_articles_stats = pd.DataFrame({
    'nb_outlinks_existants': df_valid['source_url'].value_counts(),
    'nb_inlinks_existants': df_valid['target_url'].value_counts(),
    'target_anchor_diversity': df_valid.groupby('target_url')['anchor_text'].nunique(),
    'target_mean_similarity': df_valid.groupby('target_url')['similarity_score'].mean(),
}).fillna(0).astype({'nb_outlinks_existants': int, 'nb_inlinks_existants': int, 'target_anchor_diversity': int})

df_articles_stats['target_is_orphan'] = df_articles_stats['nb_inlinks_existants'] == 0
df_articles_stats['target_depth'] = df_articles_stats.index.str.count('/') - 2  # ajuste selon ta structure d’URL

df_articles_stats = df_articles_stats.reset_index().rename(columns={'index': 'article_url'})
df_articles_stats['website'] = df_articles_stats['article_url'].str.split('https://').str[1].str.split('/').str[0]  # extraction du site à partir de l'URL
df_articles_stats['website'] = df_articles_stats['website'].str.replace(r'www', r'https://www.', regex=False)  # assure que le site commence par https://www.
df_articles_stats.to_csv(f'{output_dir}/df_articles_stats.csv', sep='|', index=False)

In [ ]:
df_articles_stats['website'].isna().sum()  # Vérification des valeurs manquantes dans la colonne 'website'

In [ ]:
df[['article_url', 'article_views']].to_csv(f'{output_dir}/df_views.csv', sep='|', index=False)

In [ ]:
# Filtrer les liens les plus forts
df_sub = df_internal_links[
    (df_internal_links['similarity_score'] > 70) &
    (df_internal_links['article_website'] == 'https://www.droit-compta-gestion.fr')
]

# Construire le graphe
G = nx.from_pandas_edgelist(df_sub, 'source_url', 'target_url', ['similarity_score'])
pos = nx.spring_layout(G, k=0.15, iterations=20)

# Préparer les arêtes avec épaisseur proportionnelle à la similarité
edge_x, edge_y, edge_width = [], [], []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    # Normaliser la similarité pour l'épaisseur
    sim = edge[2]['similarity_score']
    width = 1 + (sim - 70) / 10  # Ajuste ce facteur selon l'effet visuel souhaité
    edge_width.append(width)

# Pour Plotly, il faut une trace par épaisseur différente (astuce d’empilement)
edge_traces = []
for i, (x0, y0, x1, y1, width) in enumerate(zip(edge_x[::3], edge_y[::3], edge_x[1::3], edge_y[1::3], edge_width)):
    edge_traces.append(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        line=dict(width=width, color='#888'),
        hoverinfo='none',
        mode='lines'
    ))

# Préparer les nœuds avec taille proportionnelle au degré
node_x, node_y, node_text, node_size = [], [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    degree = G.degree(node)
    node_size.append(8 + 2 * degree)  # Ajuste le facteur selon l’effet visuel souhaité

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=node_size,
        color='blue',
        line_width=2
    )
)

fig = go.Figure(data=edge_traces + [node_trace])
fig.update_layout(
    title='Réseau de liens internes (épaisseur ∝ similarité, taille ∝ connexions)',
    showlegend=False
)
fig.show()

In [ ]:
# Filtrer les liens les plus forts
df_sub = df_internal_links[
    (df_internal_links['similarity_score'] > 70) &
    (df_internal_links['article_website'] == 'https://www.droit-compta-gestion.fr')
]

# Construire le graphe
G = nx.from_pandas_edgelist(df_sub, 'source_url', 'target_url', ['similarity_score'])
pos = nx.spring_layout(G, k=0.15, iterations=20)

# Préparer la normalisation pour la couleur des arêtes
sim_scores = [G.edges[edge]['similarity_score'] for edge in G.edges()]
norm = mcolors.Normalize(vmin=min(sim_scores), vmax=max(sim_scores))
cmap = cm.get_cmap('plasma')  # Palette continue

# Créer les traces d'arêtes avec couleur et épaisseur personnalisées
edge_traces = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    sim = edge[2]['similarity_score']
    color = mcolors.to_hex(cmap(norm(sim)))
    width = 1 + (sim - 70) / 10  # Ajuste ce facteur si besoin
    edge_traces.append(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        line=dict(width=width, color=color),
        hoverinfo='none',
        mode='lines'
    ))

# Calculer la taille des nœuds selon leur degré
node_x, node_y, node_text, node_size = [], [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    degree = G.degree(node)
    node_size.append(8 + 2 * degree)  # Ajuste le facteur selon l'effet visuel souhaité

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=node_size,
        color='blue',  # Tu peux aussi utiliser une couleur selon une autre métrique
        line_width=2
    )
)

fig = go.Figure(data=edge_traces + [node_trace])
fig.update_layout(
    title='Réseau de liens internes (épaisseur & couleur ∝ similarité, taille ∝ connexions)',
    showlegend=True
)
fig.update_layout(
    legend=dict(
        orientation="v",    # vertical (par défaut)
        yanchor="top",      # ancrage vertical en haut
        y=1,                # position verticale (1 = en haut)
        xanchor="left",     # ancrage horizontal à gauche de la légende
        x=1.05              # position horizontale (>1 pour la placer à droite du graphe)
    )
)

fig.show()

In [ ]:

# Création du graphe depuis le dataframe
G = nx.from_pandas_edgelist(df_sub, 'source_url', 'target_url', ['similarity_score'])

# Position des nœuds
pos = nx.spring_layout(G, k=0.15, iterations=20)

# Arêtes
edge_x, edge_y = [], []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Nœuds
node_x, node_y, node_text = [], [], []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)  # ici tu peux personnaliser le texte affiché

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        color=[],
        size=10,
        colorbar=dict(title='Nombre de connexions'),
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Réseau de similarité des articles',
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
               )

fig.show()

In [ ]:
# Nouvelle fonction améliorée pour suggérer des ancres courtes
def suggérer_ancres_courtes(source_text, target_termes, stop_words, n_max=5, longueur_max=4):
    """
    Suggère des groupes nominaux courts issus du texte source en identifiant ceux qui
    contiennent au moins un terme significatif du document cible.
    
    - n_max : nombre maximum d’ancres retournées
    - longueur_max : nombre maximum de mots par ancre (groupe nominal)
    """
    ancres_possibles = set()
    doc = nlp(source_text)

    for chunk in doc.noun_chunks:
        tokens = [t.lemma_.lower() for t in chunk if not t.is_stop and t.is_alpha]
        if not tokens or len(tokens) > longueur_max:
            continue
        if set(tokens) & target_termes:
            ancre = " ".join(tokens)
            if all(t not in stop_words for t in tokens):
                ancres_possibles.add(ancre)
        if len(ancres_possibles) >= n_max:
            break

    return list(ancres_possibles)


In [ ]:
# 1. Filtrage sur le site concerné
mask = df_termes['website'].isin(selected_websites)
df_termes_selected = df_termes[mask]

df_links_selected = df_links[
    (df_links['article_website'].isin(selected_websites)) &
    (df_links['target_url'].apply(lambda url: any(url.startswith(site) for site in selected_websites)))
]

# 2. Dictionnaire {article_url: set(termes)}
df_termes_grouped_dcg = df_termes_selected.groupby("article_url")["terme"].apply(set).to_dict()

# 3. Génération des ancres suggérées
anchor_suggestions = []

for _, row in tqdm(df_links_selected.iterrows(), total=len(df_links_selected), desc="Génération des ancres courtes"):
    source_url = row['source_url']
    target_url = row['target_url']

    # Récupération du texte de l’article source
    source_text_series = df[df['article_url'] == source_url]['article_content']
    if source_text_series.empty:
        anchor_suggestions.append([])
        continue

    source_text = str(source_text_series.values[0])
    target_termes = df_termes_grouped_dcg.get(target_url, set())

    ancres = suggérer_ancres_courtes(source_text, target_termes, stop_words_fr)
    anchor_suggestions.append(ancres)

df_links_selected['anchor_suggestions'] = anchor_suggestions

In [ ]:
# Liste des articles sources ayant eu des suggestions
sources_avec_proposition = set(df_links_selected['source_url'].unique())

# Liste de tous les articles du site
tous_les_articles = set(df[df['website'] == 'https://www.droit-compta-gestion.fr']['article_url'].unique())

# Quelles sources sont absentes ?
sources_sans_proposition = tous_les_articles - sources_avec_proposition

print(f"Nombre d'articles sans suggestion : {len(sources_sans_proposition)}")
# Et pour voir lesquels :
list(sources_sans_proposition)

In [ ]:
df_links_selected[~df_links_selected['lien_existant']]

In [ ]:
df_exploded = df_links_selected.copy()
df_exploded = df_exploded.explode('anchor_suggestions')
df_exploded

In [ ]:
df_exploded.to_csv(f'{output_dir}/df_links_exploded.csv', sep='|', index=False)